In [91]:
import sys
import matplotlib

ModuleNotFoundError: No module named 'matplotlib'

In [92]:
# highly dependent on specific output format from data.sh
# not at all flexible 
def parsedata(filename):
    
    # create list of tuple w/ coordinates for graph
    f = open(filename, "r")
    coordinates = []
    point = [0.0, 0.0]
    for line in f.readlines():
        line = line.split()
        #if line[0] == "SCALE":        # scale 
        #    point[0] = float(line[1])
        if line[0].isnumeric():        # iteration count
            point[0] = float(line[0])
        elif line[0] == "Average":     # average execution time
            point[1] = float(line[2])
            coordinates.append(tuple(point))
    
    f.close()

    print("iter, Avg. Time")
    for t in coordinates:
        print(t)
    
    return coordinates

In [ ]:
def graph(data):
    pass
    # matplotlib?

In [93]:
def main():
    
    # pass output text from data.sh script to parsedata()
    gs_data = parsedata("18-27/gaussseidellPARALLEL.txt")
    jc_data = parsedata("18-27/jacobiPARALLEL.txt")

    
if __name__ == "__main__":
    main()

iter, Avg. Time
(16.0, 0.00626)
(13.0, 0.0065)
(11.0, 0.01348)
(12.0, 0.03161)
(11.0, 0.07203)
(6.0, 0.09919)
(5.0, 0.1946)
(5.0, 0.47223)
(4.0, 1.01893)
(4.0, 2.70616)
(4.0, 8.26872)
iter, Avg. Time
(7.0, 0.00244)
(7.0, 0.00409)
(7.0, 0.00893)
(7.0, 0.02346)
(6.0, 0.04615)
(6.0, 0.12508)
(6.0, 0.22815)
(6.0, 0.54932)
(6.0, 3.38666)
(6.0, 5.01225)
(6.0, 17.09382)
